<a href="https://colab.research.google.com/github/movie112/Git/blob/main/210901_210907/%5BPaper_Review%5D_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Paper Review] FastText: Enriching Word Vectors with Subword Information
- [논문](https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00051/43387/Enriching-Word-Vectors-with-Subword-Information)
- [참고](https://youtu.be/7UA21vg4kKE)


---
## Overview

### FastText
- 자연어를 vector로 표현하는 word representation의 방법론
- baseline(CBOW, Skip-gram) 대비 빠른 학습 속도
- subword: morpology(형태론)적 feature 정보를 한 단어의 subword 즉, character level에서 추출
---

## Prerequisite

### word embedding
- word representation = word embedding 혼용


### Word2vec
- 기존 neural net 기반의 representation 한계점 개선
  - non-linear hidden layer 제거 -> 구조 단순화, 연산 복잡성 개선
  - distributed representation 제안(CBOW, Skip-gram) -> 보다 많은 데이터 효울적으로 학습 가능
-  학습된 word vector는 연산 가능
  - vec(king) - vec(man) + vec(woman) = vec(queen)
- Skip-gram이 CBOW보다 상대적으로 성능 좋음
  - 역전파의 gradient flow가 CBOW는 context vector 만큼 분산, Skip-gram은 주변 정보 합산
- corpus 크기에 따라 느려지는 학습 속도 개선
  - hierarchical softmax, negative sampling 기법 등이 함께 사용
---


## Paper Review

### 1) introduce

#### [기존 representation 방법의 한계점]
1. 모든 단어 각각을 vector로 1:1 representation 하는 것은 한계 존재
  - training data 에 등장하지 않은 rare word는 정확한 embedding 어렵 : OOV 문제

> tensor -> Word2vec -> O   
> flow -> Word2vec -> O   
> tensorflow -> X    

2. 단어 자체의 내부적 구조룰 무시함
  - morphologically rich language를 표현하는 것에 한계
    - morphologically rich language: 다양한 조사나 어미 등의 기능어가 내용어와 결합
  - parameter 공유 없이 유사한 의미의 여러 형태의 단어를 모두 representation하는 것은 어렵
  - 유사한 의미의 개별적 단어가 context 정보를 기반으로 unique하게 학습 되는 문제
    - eat, eats, eaten, eater, eating

#### [FastText의 제안]
- paraphrase pairs 기반 학습이 아닌, 어떠한 text corpus 학습에도 효율적이면서 형태론적으로 의미 있는 representation 방법 제안
  - extension of the continuous skip-gram model
  - character level information by character n-gram
---

### 2) Model Architecture

    

#### [General model] - base model for FastText
- 기본적으로 차용하고 있는 모델 및 기법
   - Skip-gram
   - Negative Sampling
  
- Skip-gram 
  - 중심 단어로부터 주변단어를 예측하는 방향으로 학습    
  - 주어진 window 크기로 sliding하면서 중심단어 별로 각 주변단어의 확률값 업데이트
  - (한계점)
    - 비효율적인 연산: corpus의 dimension 대비 실질적으로 업데이트 계산에 사용되는 부분은 일부

- Negative Sample
  - 이러한 문제를 해결하는 기존의 방법
  - input의 단어들은 one-hot index니까, 가중치 벡터의 row index를 그냥 
- 발전
  - positive example: target word의 실제 context word
  - nagative example: 실제 context word 아닌 일정한 확률값으로 뽑힌 word
    - multi-class claaification 문제를 binary classification 문제로 치환: context word=1, 아닌 거=0 부여
    - target word와 positive, negative examples의 index를 가지고 연산 수행 -> 연산 복잡도 줄임
  - corpus 내 자주 등장하는 단어를 더 많이 추출하고 드물게 등장하는 단어는 적게 추출하고자 함

#### [Subword model] - methodological improvement in FastText
- 기존 representation models의 문제점
  - 모든 단어 1:1 representation -> rare word 학습 어려움
  - 형태론적 의미 정보 활용 못함
    - 내부적으로 구조가 유사한 inflected form compund word 등에 대해 prarmeter 공유 없이 각자 unique하게 학습

- FastText는 parameter 공유로 subword information을 담을 수 있는 방법론 제안
  - subword: 어떤 단어를 글자 단위로 쪼갬
  - 흔히 쓰이는 단어(Freauent word)와 특정 문서에만 나오는 단어(Rare word)로 구분할 수 있는데, 형태론적으로 쪼갰을 때 이 둘이 공통되는 부분이 있다면 공유되는 글자 단위를 바탕으로 의미를 유추 가능
    - 벡터화 관점에서는 비슷한 representation으로 인코딩 가능
  - 즉, 공통되는 형태소들에 대해서 parameter share 함으로써 embedding에 반영 가능(의미 공유 가능), 심지어 rare/unseen word에 대해서도 쉽게 표현 가능

- fasttext의 subword generation
  - '<', '>'을 추가하여 단어의 시작, 끝을 표현
  - n-gram의 n은 character 개수(<도 포함)

- 발전
  - use hashing function for memory efficiency
    - 데이터가 클 수록 unique n-gram의 가지수도 증가
    - hasing function으로 n-gram을 `1~K의 integer`로 mapping 후 `1~B 사이의 hash` 값 계산
    - 해당 hash 값을 index로 하여 hashed dictionary로 불리는 n-gram 간의 공유 matrix에서 vector 뽑아 사용
      - ing(n-gram) -> hasing function -> 10

  - shared embedding의 사용으로 메모리 효율 높임
  - hash 값의 충돌이 일어날 수 있으나, vocab의 사이즈를 제어하는 데 효율적
  - 본 논문에서는 B를 약 2 million 
---


### 3) Experiment 


#### [result]

- 6가지 관점으로 실험 진행
1. Spearman's rank correlation
  - sisg: subword information skip-gram, OOV도 n-gram의 sum으로 표현
  - sisg-: OOV를 null vector로 사용
    - OOV를 charaacter n-gram으로 표현했을 때의 효과 입증
    - 문법적 기능에 따라 형태가 변하는 단어, 합성어가 많은 German, Russian에 대해 상대적으로 n-gram 효과가 큼
    - English WS353는 common word가 많은 dataset이었기 때문에 n-gram의 효과가 크지 않았음

2. Word Analogy Task
  - syntactic task에 좋음
  - semantic에는 개선 효과 없음 -> 적절한 n값 선택으로 성능 향상 가능

3. comoparision with morphological representation
  - 다른 형태론적 접근 모델에 비해 fasttext의 성능이 좋음
  - 다른 모델: 합성어와 같은 복합적인 단어는 제대로 modeling 못함, fasttext는 가능

4. effect of the size of training data
  - 모든 dataset 크기에 대해 sisg가 outperformed
  - scalability문제 -> dataset 크기 증가가 성능 증가에 무한히 비례하는 것 아님, 포화 상태
  - sisg는 적은 데이터셋만으로도 충분히 Outperformed

5. effect of the size of n-grams
  - n 이 5 또는 6이하 & 3 이상에서 성능 향상
    - n-gram 생성시, 단어 앞 뒤로 '<', '>'가 들어가기 떄문에 2(bi)-gram은 큰 의미 없음
    - test data의 scarcity을 생각하며 적당한 vaildation precedure로 합리적인 n 조정 필요

6. language modeling task based on LSTM
  - 제안된 모델의 성능이 제일 좋음, 특히 morphologically rich lanquages 에 대해
    - LSTM: 기본 LSTM /  sg: LSTM+skip-gram / sisg: LSTM+n-gram+skip-gram
---

## Summary
- character n-gram 과 skip-gram을 함께 사용하는 것의 성능이 좋음
- fast & simple method
- preprocessing, supervision 없이 어떠한 text라도 Representation 가능